In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import streamlit as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:

# Load and merge datasets
env_data = pd.read_csv('gs://draining-and-test-dataset/Winquist_vcov.csv')
patient_data = pd.read_csv('gs://draining-and-test-dataset/asthma_disease_data.csv')
aqi_data = pd.read_csv('gs://draining-and-test-dataset/global air pollution dataset.csv')


In [3]:
# Clean column names
env_data.columns = env_data.columns.str.strip()
aqi_data.columns = aqi_data.columns.str.strip()
patient_data.columns = patient_data.columns.str.strip()

In [4]:
print(env_data.columns.tolist())


['Effect', 'Model Specification', 'Season', 'Pollutant', 'O3', 'CO', 'NO2', 'SO2', 'PM2.5', 'EC', 'SO42-', 'Secondary PM2.5 a', 'CO*NO2', 'CO*EC', 'EC*NO2', 'NO2*O3', 'O3*SO2', 'NO2*SO2', 'O3*Secondary PM2.5 a', 'SO2*SO42-', 'CO*O3', 'O3*PM2.5', 'CO*SO2', 'CO*PM2.5', 'NO2*PM2.5', 'PM2.5*SO2']


In [5]:

# Feature Engineering for each dataset independently
def preprocess_env_data(df):
    df['Pollution_Index'] = (df['PM2.5']/35) + (df['NO2']/100) + (df['O3']/70)
    return df[['Pollutant', 'PM2.5', 'NO2', 'O3', 'Pollution_Index']]

def preprocess_aqi_data(df):
    df['AQI_Health_Risk'] = (df['PM2.5 AQI Value']/50) + (df['NO2 AQI Value']/100)
    return df[['City', 'AQI Value', 'AQI Category', 'AQI_Health_Risk']]

def preprocess_patient_data(df):
    df['Health_Risk_Score'] = df['PollutionExposure'] * df['LungFunctionFEV1']
    return df[['PatientID', 'Age', 'BMI', 'Smoking', 'PollutionExposure',
               'LungFunctionFEV1', 'Health_Risk_Score', 'Diagnosis']]


env_features = preprocess_env_data(env_data)
aqi_features = preprocess_aqi_data(aqi_data)
patient_features = preprocess_patient_data(patient_data)

# Merge datasets based on independent modeling needs (no direct relationship)
merged_features = pd.concat([env_features, aqi_features, patient_features], axis=1)



In [6]:
numeric_columns = merged_features.select_dtypes(include=['number']).columns
merged_features[numeric_columns] = merged_features[numeric_columns].fillna(merged_features[numeric_columns].mean())
non_numeric_columns = merged_features.select_dtypes(exclude=['number']).columns
for col in non_numeric_columns:
    merged_features[col].fillna(merged_features[col].mode()[0], inplace=True)
# Identify numeric and non-numeric columns
numeric_columns = merged_features.select_dtypes(include=['number']).columns
non_numeric_columns = merged_features.select_dtypes(exclude=['number']).columns

# Fill missing values for numeric columns with mean
merged_features[numeric_columns] = merged_features[numeric_columns].fillna(merged_features[numeric_columns].mean())


/var/tmp/ipykernel_3433/4227718648.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_features[col].fillna(merged_features[col].mode()[0], inplace=True)


In [7]:

# Fill missing values for non-numeric columns with mode
for col in non_numeric_columns:
    merged_features[col].fillna(merged_features[col].mode()[0], inplace=True)

print("Missing values handled successfully!")


Missing values handled successfully!


/var/tmp/ipykernel_3433/1443434359.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_features[col].fillna(merged_features[col].mode()[0], inplace=True)


In [20]:
numeric_columns = merged_features.select_dtypes(include=['number']).columns
merged_features[numeric_columns] = merged_features[numeric_columns].fillna(merged_features[numeric_columns].mean())
non_numeric_columns = merged_features.select_dtypes(exclude=['number']).columns
numeric_columns = merged_features.select_dtypes(include=['number']).columns
non_numeric_columns = merged_features.select_dtypes(exclude=['number']).columns
for col in non_numeric_columns:
    merged_features[col] = merged_features[col].fillna(merged_features[col].mode()[0])


In [21]:
# Handle missing values
merged_features.fillna(merged_features.mean(), inplace=True)

# Split data for training and testing
X = merged_features.drop(columns=['Diagnosis'])
y = merged_features['Diagnosis']

TypeError: Could not convert ['O3CONO2SO2PM2.5ECSO42-Secondary PM2.5 aO3CONO2SO2PM2.5ECSO42-Secondary PM2.5 aO3NO2SO2O3NO2SO2O3Secondary PM2.5 aO3Secondary PM2.5 aO3CONO2SO2PM2.5O3CONO2SO2PM2.5CONO2ECCONO2ECSO2SO42-SO2SO42-CONO2ECCO*NO2CO*ECEC*NO2CONO2ECCO*NO2CO*ECEC*NO2O3NO2SO2NO2*O3O3*SO2NO2*SO2O3NO2SO2NO2*O3O3*SO2NO2*SO2O3Secondary PM2.5 aO3*Secondary PM2.5 aO3Secondary PM2.5 aO3*Secondary PM2.5 aSO2SO42-SO2*SO42-SO2SO42-SO2*SO42-O3CONO2SO2PM2.5CO*O3NO2*O3O3*SO2O3*PM2.5CO*NO2CO*SO2CO*PM2.5NO2*SO2NO2*PM2.5PM2.5*SO2O3CONO2SO2PM2.5CO*O3NO2*O3O3*SO2O3*PM2.5CO*NO2CO*SO2CO*PM2.5NO2*SO2NO2*PM2.5PM2.5*...'] to numeric

In [19]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=200, max_depth=6, random_state=42)
#model = RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42)
model.fit(X_train, y_train)


ValueError: could not convert string to float: 'NO2'

In [ ]:
'''
# Evaluate model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))'''

# Evaluate model
y_pred = model.predict(X_test)

# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation results
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

In [ ]:

# Save trained model
joblib.dump(model, 'asthma_prediction_model.pkl')


In [ ]:
'''
# Streamlit UI for risk prediction
def main():
    st.title("Asthma Risk Prediction System")

    # Sidebar inputs for prediction
    with st.sidebar:
        st.header("Input Parameters")
        age = st.slider('Age', 5, 80, 30)
        bmi = st.number_input('BMI', 15.0, 40.0, 22.0)
        smoking_status = st.selectbox('Smoking Status', ['Non-Smoker', 'Occasional Smoker', 'Frequent Smoker'])
        pollution_exposure = st.slider('Pollution Exposure Level (%)', 0, 100, 50)
        lung_function_fev1 = st.slider('Lung Function FEV1 (%)', 50, 150, 100)
        pm25_level = st.slider('PM2.5 Level (µg/m³)', 0, 500, 35)
        no2_level = st.slider('NO2 Level (ppb)', 0, 200, 40)
        o3_level = st.slider('Ozone Level (ppb)', 0, 200, 50)

    # Calculate derived features for prediction
    pollution_index = (pm25_level/35) + (no2_level/100) + (o3_level/70)
    health_risk_score = pollution_exposure * lung_function_fev1

    input_data = [[age, bmi, pollution_exposure, lung_function_fev1,
                   pm25_level, no2_level, o3_level,
                   pollution_index, health_risk_score]]

    # Load trained model
    model = joblib.load('asthma_prediction_model.pkl')

    # Predict risk level using the trained model
    risk_prediction = model.predict(input_data)[0]
    risk_probability = model.predict_proba(input_data)[0][1] * 100

    # Display results
    st.subheader("Prediction Results")
    st.metric("Asthma Risk Diagnosis", risk_prediction)
    st.metric("Risk Probability (%)", f"{risk_probability:.2f}")

    st.write("""
    ### Recommended Actions:
    - Avoid outdoor activities when PM2.5 > 35 µg/m³.
    - Use N95 masks in high pollution areas.
    - Maintain indoor air quality with purifiers.
    - Regular lung function checks.
    """)

if __name__ == "__main__":
    main()'''

In [ ]:
import streamlit as st
import joblib

# Streamlit UI for risk prediction
def main():
    st.title("Asthma Risk Prediction System")

    # Sidebar inputs for prediction
    with st.sidebar:
        st.header("Input Parameters")
        age = st.slider('Age', 5, 80, 30)
        bmi = st.number_input('BMI', 15.0, 40.0, 22.0)
        smoking_status = st.selectbox('Smoking Status', ['Non-Smoker', 'Occasional Smoker', 'Frequent Smoker'])
        pollution_exposure = st.slider('Pollution Exposure Level (%)', 0, 100, 50)
        lung_function_fev1 = st.slider('Lung Function FEV1 (%)', 50, 150, 100)
        pm25_level = st.slider('PM2.5 Level (µg/m³)', 0, 500, 35)
        no2_level = st.slider('NO2 Level (ppb)', 0, 200, 40)
        o3_level = st.slider('Ozone Level (ppb)', 0, 200, 50)

    # Calculate derived features for prediction
    pollution_index = (pm25_level / 35) + (no2_level / 100) + (o3_level / 70)
    health_risk_score = pollution_exposure * lung_function_fev1

    '''input_data = [[age, bmi, pollution_exposure, lung_function_fev1,
                   pm25_level, no2_level, o3_level,
                   pollution_index, health_risk_score]]'''

# Define the input data
# Define the input data based on the expected features
    input_data = [
    [
        # Environmental data (example values)
        35,  # PM2.5 level (µg/m³)
        40,  # NO2 level (ppb)
        50,  # O3 level (ppb)
        0.5,  # CO level (ppm)
        0.2,  # SO2 level (ppm)
        10,  # EC level (µg/m³)
        5,  # SO42- level (µg/m³)
        0.1,  # CO*NO2 level (ppm*ppb)
        0.05,  # CO*EC level (ppm*µg/m³)
        0.01,  # EC*NO2 level (µg/m³*ppb)
        0.005,  # NO2*O3 level (ppb*ppb)
        0.002,  # O3*SO2 level (ppb*ppm)
        0.0005,  # O3*Secondary PM2.5 a level (ppb*µg/m³)
        0.0002,  # SO2*SO42- level (ppm*µg/m³)
        0.0001,  # CO*O3 level (ppm*ppb)
        0.00005,  # O3*PM2.5 level (ppb*µg/m³)
        
        ]
    ]
    
    # Load trained model
    model = joblib.load('asthma_prediction_model.pkl')

    # Predict risk level using the trained model
    risk_prediction = model.predict(input_data)[0]

    # Display results
    st.subheader("Prediction Results")
    st.metric("Predicted Health Risk Score", f"{risk_prediction:.2f}")

    st.write("""
    ### Recommended Actions:
    - Avoid outdoor activities when PM2.5 > 35 µg/m³.
    - Use N95 masks in high pollution areas.
    - Maintain indoor air quality with purifiers.
    - Regular lung function checks.
    """)

if __name__ == "__main__":
    main()